In [1]:
import glob
import json
from pathlib import Path

files = glob.glob("outputs/*.json")
chats = {}
print(files)
for f in files:
    with open(f) as filein:
        chats[Path(f).stem] = json.load(filein)

['outputs/trial_01.json', 'outputs/trial_04.json', 'outputs/trial_05.json', 'outputs/trial_02.json', 'outputs/trial_03.json']


In [4]:
from itertools import zip_longest
from models import *

lines = []
for trial_n, chat in chats.items():
    for question in chat:
        answers = [Answer(**a) for a in question["answers"]]
        source_texts = [SourceText(**t) for t in question["source_texts"]]
        zipped = zip_longest(source_texts, answers)
        q_n = question["idx"] + 1
        q = question["question"]
        for source_text, answer in zipped:
            lines.append((q_n, q, source_text, answer))

In [14]:
import pandas as pd
# import dataclasses

df = pd.DataFrame(lines, columns=["Question #", "Question", "Source Text", "Answer"])
df["Source Text"] = df["Source Text"].apply(
    lambda x: x.value if isinstance(x, SourceText) else x
)
df["Answer"] = df["Answer"].apply(lambda x: x.value if isinstance(x, Answer) else x)
df.to_csv('source_&_answers.csv', index=False)